# Format raw and processed files for GEO uploads

Note: This is written to be run on a Mac with the research drive mounted. Adjust file paths as necessary if this is not the case.

## Setup

In [12]:
from collections import namedtuple
import pandas as pd
import os
import hashlib

# Setting up this named tuple to store any further information we need about the runs
# (such as directory paths, etc.)
RunInfo = namedtuple("RunInfo", ["name", "fastq_folder"])

runs = [
    RunInfo("Run220824", "/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220824_VL00320_14_AAC7WHVM5/Analysis/1/Data/fastq"),
    RunInfo("Run220825", "/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220825_VL00320_15_AAC5JYKM5/Analysis/1/Data/fastq"),
    RunInfo("Run220826", "/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220826_VL00320_16_AAC5K32M5/Analysis/1/Data/fastq"),
]

experiment_tag = "220829_VL00320_AAC7WHVM5_AAC5JYKM5_AAC5K32M5"

# Path to processed data files
processed_data_folder = f"/Volumes/kueck/Pipeline/CellLines/RNAseq/{experiment_tag}/star_salmon"
processed_data_file_names = [
    "salmon.merged.gene_tpm.tsv",
    "salmon.merged.gene_counts.tsv",
]

# Path to fastq folder where all the fastq files will be copied to
# Note that this copies straight into the research drive so that the fastq files are not on local device
final_fastq_folder = f"/Volumes/kueck/ovarian_cancer_cisplatin_response_manuscript/geo_metadata/data/{experiment_tag}/fastq"

# Paths to save the sample/processed data info
sample_info_path = f"../data/{experiment_tag}/raw_sample_info.csv"
processed_data_info_path = f"../data/{experiment_tag}/processed_data_info.csv"
formatted_sample_files_path = f"../data/{experiment_tag}/raw_sample_files.csv"

# Print command and run command
def run_command(cmd):
    print(f"cmd: {cmd}")
    os.system(cmd)

# DataFrame to hold info about each sample
sample_info = pd.DataFrame(
    columns=["sample_id", "original_fastq_path", "symlink_fastq_path", "final_file_name", "md5_checksum"]
)
sample_info.set_index("sample_id", inplace=True)

In [23]:
def compute_md5_checksum(file_path):
    md5_hash = hashlib.md5()
    with open(file_path, "rb") as file:
        # Read the file in chunks to avoid memory issues with large files
        for chunk in iter(lambda: file.read(4096), b""):
            md5_hash.update(chunk)
    return md5_hash.hexdigest()

## Collect information about each sample

In [ ]:
for run in runs:
    for sample_num in range(1, 37):
        for read_num in range(1, 3):
            # Infer the original fastq path
            original_fastq_path = f"{run.fastq_folder}/{sample_num}_S{sample_num}_R{read_num}_001.fastq.gz"

            # Create a unique sample_id including the run tag
            sample_id = f"{sample_num}_S{sample_num}_R{read_num}_001_{run.name}"

            # Add sample info to data frame
            sample_info.loc[sample_id] = [original_fastq_path, "", ""]

display(sample_info)

## Compute MD5 checksums

In [ ]:
i = 1
for sample_id, row in sample_info.iterrows():
    display(f"Computing MD5 for {sample_id} ({i}/{len(sample_info)})")
    sample_original_path = row["original_fastq_path"]

    # Compute md5 checksum
    md5_checksum = compute_md5_checksum(sample_original_path)

    # Update sample info
    sample_info.at[sample_id, "md5_checksum"] = md5_checksum

    i += 1

display(sample_info)

## Make a symlink to fastq files

In [ ]:
i = 1
for sample_id, row in sample_info.iterrows():
    display(f"Making symlink for fastq file {sample_id} ({i}/{len(sample_info)})")

    # Create symbolic link
    sample_original_path = row["original_fastq_path"]
    final_file_name = f"{sample_id}.fastq.gz"
    symlink_fastq_path = f"{final_fastq_folder}/{sample_id}.fastq.gz"
    os.symlink(sample_original_path, symlink_fastq_path)

    # Update sample info
    sample_info.at[sample_id, "symlink_fastq_path"] = symlink_fastq_path
    sample_info.at[sample_id, "final_file_name"] = final_file_name

    i += 1

display(sample_info)

In [ ]:
# Code for deep copy:

# i = 1
# for sample_id, row in sample_info.iterrows():
#     display(f"Copying fastq file {sample_id} ({i}/{len(sample_info)})")
#     sample_original_path = row["original_fastq_path"]
#     copied_fastq_path = f"{final_fastq_folder}/{sample_id}.fastq.gz"

#     # Copy the fastq file
#     run_command(f"cp {sample_original_path} {copied_fastq_path}")

#     # Compute md5 checksum on copied file
#     display(f"Computing MD5 for file {sample_id} ({i}/{len(sample_info)})")
#     copied_md5_hash = compute_md5_checksum(copied_fastq_path)

#     # Check md5 checksum matches
#     assert (copied_md5_checksum == row["md5_checksum"], f"MD5 checksums do not match for {sample_id}")

#     # Update sample info
#     sample_info.at[sample_id, "copied_fastq_path"] = copied_fastq_path

#     i += 1
#     break

# display(sample_info)

## Save sample info

This can then be copied into the GEO metadata spreadsheet

In [25]:
sample_info.to_csv(sample_info_path, sep="\t", index=True)
display(sample_info)

,original_fastq_path,copied_fastq_path,md5_checksum,final_file_name
sample_id,,,,
1_S1_R1_001_Run220824,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220824_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,fcc7497229dd2dea1649f0bde304536d,1_S1_R1_001_Run220824.fastq.gz
1_S1_R2_001_Run220824,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220824_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,386a54cd4b9865edc92cad80a678a610,1_S1_R2_001_Run220824.fastq.gz
2_S2_R1_001_Run220824,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220824_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,81f60f95136d5a580fff230ebb8b9d5a,2_S2_R1_001_Run220824.fastq.gz
2_S2_R2_001_Run220824,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220824_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,d3a20754b0c80703877893b22e58826c,2_S2_R2_001_Run220824.fastq.gz
3_S3_R1_001_Run220824,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220824_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,ccec03142bc500ea4ec2f2da236ebb60,3_S3_R1_001_Run220824.fastq.gz
...,...,...,...,...
34_S34_R2_001_Run220826,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220826_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,1f602edda2fb77f3db0bb7f12594bcfe,34_S34_R2_001_Run220826.fastq.gz
35_S35_R1_001_Run220826,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220826_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,fc154a8a45fdb8bfe6f2beb362035de6,35_S35_R1_001_Run220826.fastq.gz
35_S35_R2_001_Run220826,/Volumes/kueck/FASTQ/CellLines/RNA-Seq/220826_...,/Volumes/kueck/ovarian_cancer_cisplatin_respon...,5c35c8aebadf7d992d6be72991dedcf7,35_S35_R2_001_Run220826.fastq.gz


## Create formatted table with files per experimental sample

In [31]:
columns = ["sample_number"]
for run in runs:
    for read_num in range(1, 3):
        columns.append(f"{run.name}_R{read_num}")

formatted_sample_info = pd.DataFrame(columns=columns)
formatted_sample_info.set_index("sample_number", inplace=True)

for sample_num in range(1, 37):
    for run in runs:
        for read_num in range(1, 3):
            column_name = f"{run.name}_R{read_num}"
            file_name = (
                f"{sample_num}_S{sample_num}_R{read_num}_001_{run.name}.fastq.gz"
            )

            # Check that this file name appears in the sample_info data frame
            assert file_name in sample_info["final_file_name"].values

            formatted_sample_info.at[sample_num, column_name] = file_name

display(formatted_sample_info)

formatted_sample_info.to_csv(formatted_sample_files_path, sep="\t", index=True)

,Run220824_R1,Run220824_R2,Run220825_R1,Run220825_R2,Run220826_R1,Run220826_R2
sample_number,,,,,,
1,1_S1_R1_001_Run220824.fastq.gz,1_S1_R2_001_Run220824.fastq.gz,1_S1_R1_001_Run220825.fastq.gz,1_S1_R2_001_Run220825.fastq.gz,1_S1_R1_001_Run220826.fastq.gz,1_S1_R2_001_Run220826.fastq.gz
2,2_S2_R1_001_Run220824.fastq.gz,2_S2_R2_001_Run220824.fastq.gz,2_S2_R1_001_Run220825.fastq.gz,2_S2_R2_001_Run220825.fastq.gz,2_S2_R1_001_Run220826.fastq.gz,2_S2_R2_001_Run220826.fastq.gz
3,3_S3_R1_001_Run220824.fastq.gz,3_S3_R2_001_Run220824.fastq.gz,3_S3_R1_001_Run220825.fastq.gz,3_S3_R2_001_Run220825.fastq.gz,3_S3_R1_001_Run220826.fastq.gz,3_S3_R2_001_Run220826.fastq.gz
4,4_S4_R1_001_Run220824.fastq.gz,4_S4_R2_001_Run220824.fastq.gz,4_S4_R1_001_Run220825.fastq.gz,4_S4_R2_001_Run220825.fastq.gz,4_S4_R1_001_Run220826.fastq.gz,4_S4_R2_001_Run220826.fastq.gz
5,5_S5_R1_001_Run220824.fastq.gz,5_S5_R2_001_Run220824.fastq.gz,5_S5_R1_001_Run220825.fastq.gz,5_S5_R2_001_Run220825.fastq.gz,5_S5_R1_001_Run220826.fastq.gz,5_S5_R2_001_Run220826.fastq.gz
6,6_S6_R1_001_Run220824.fastq.gz,6_S6_R2_001_Run220824.fastq.gz,6_S6_R1_001_Run220825.fastq.gz,6_S6_R2_001_Run220825.fastq.gz,6_S6_R1_001_Run220826.fastq.gz,6_S6_R2_001_Run220826.fastq.gz
7,7_S7_R1_001_Run220824.fastq.gz,7_S7_R2_001_Run220824.fastq.gz,7_S7_R1_001_Run220825.fastq.gz,7_S7_R2_001_Run220825.fastq.gz,7_S7_R1_001_Run220826.fastq.gz,7_S7_R2_001_Run220826.fastq.gz
8,8_S8_R1_001_Run220824.fastq.gz,8_S8_R2_001_Run220824.fastq.gz,8_S8_R1_001_Run220825.fastq.gz,8_S8_R2_001_Run220825.fastq.gz,8_S8_R1_001_Run220826.fastq.gz,8_S8_R2_001_Run220826.fastq.gz
9,9_S9_R1_001_Run220824.fastq.gz,9_S9_R2_001_Run220824.fastq.gz,9_S9_R1_001_Run220825.fastq.gz,9_S9_R2_001_Run220825.fastq.gz,9_S9_R1_001_Run220826.fastq.gz,9_S9_R2_001_Run220826.fastq.gz


In [30]:
display(formatted_sample_files_path)

'../data/220829_VL00320_AAC7WHVM5_AAC5JYKM5_AAC5K32M5/raw_sample_files.csv'

## Compute MD5 Checksums for processed data files

In [32]:
processed_file_info = pd.DataFrame(
    columns=["file_name", "md5_checksum"]
)

for file_name in processed_data_file_names:
    file_path = f"{processed_data_folder}/{file_name}"

    # Compute md5 checksum
    md5_checksum = compute_md5_checksum(file_path)

    # Add to processed file info
    processed_file_info.loc[file_name] = [file_name, md5_checksum]

display(processed_file_info)

# Save processed data info
processed_file_info.to_csv(processed_data_info_path, sep="\t", index=False)

,file_name,md5_checksum
salmon.merged.gene_tpm.tsv,salmon.merged.gene_tpm.tsv,9514c74be45f5da5c5141481f55a1116
salmon.merged.gene_counts.tsv,salmon.merged.gene_counts.tsv,1a6563a34960d99b22f978dd8210e266
